# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097001


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:36,  3.43s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:05,  2.43s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.75s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.28s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:01,  3.90it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.30it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.27it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  3.69it/s]

ElisabethB                            0.000098
not-logged-in-3b0b929f4475fd276561    0.017868
not-logged-in-8eae5b2dbfb8c457c47a    0.000170
SuperstringMuon                       0.000132
Quarkgc                               0.000143
jnarayanbvg                           0.000144
not-logged-in-db23dc032fc427a4e4d9    0.000143
boomod                                0.000196
rainidaygirl                          0.000286
awright5                              0.000273
jmartinez468                          0.004241
not-logged-in-75caf8064c4ee245413e    0.011179
withadnotat                           0.000139
merkosh                               0.000307
hopefuji                              0.000253
ShirleyL                              0.000152
not-logged-in-04ed9ae880f18762cd9e    0.004084
Lavadude                              0.025803
layne.woodward2701                    0.000174
w7250369                              0.000173
Anfrily                               0.004446
Denny2507    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())